In [17]:
import pandas as pd
import numpy as np
import os

In [18]:
# read in upd_gen csv
upd_gen_df = pd.read_csv('a_upd_generator_df.csv')
# read in data_source_comparisons
manual_db_rel = pd.read_csv(os.path.join('..', 'data','manual_db_rel.csv'))

In [19]:
# the data that should exist for RICE is:
reqd_thermal_data = [
                  'Resource',
                  'Zone',
                  'Model',
                  'New_Build',
                  'Can_Retire',
                  'Existing_Cap_MW',
                  'Max_Cap_MW',
                  'Min_Cap_MW',
                  'Inv_Cost_per_MWyr',
                  'Fixed_OM_Cost_per_MWyr',
                  'Var_OM_Cost_per_MWh',
                  'Heat_Rate_MMBTU_per_MWh',
                  'Fuel',
                  'Cap_Size',
                  'Start_Cost_per_MW',
                  'Start_Fuel_MMBTU_per_MW',
                  'Up_Time',
                  'Down_Time',
                  'Ramp_Up_Percentage',
                  'Ramp_Dn_Percentage',
                  'Min_Power',
                  'Reg_Max',
                  'Rsv_Max',
                  'Reg_Cost',
                  'Rsv_Cost',
                  'region',
                  'cluster',
                  ]

In [20]:
# get list of 'Resources' in manual_db_rel that are thermal 'Energy Type'
thermal_resources = manual_db_rel[manual_db_rel['Energy Type'] == 'Thermal']['Resource'].tolist()

# get dataframe from upd_gen_df that has 'Resource' in thermal_resources
thermal_df = upd_gen_df[upd_gen_df['Resource'].isin(thermal_resources)]


In [21]:
# extract thermal data of required columns
reqd_thermal_df = thermal_df[reqd_thermal_data]

In [22]:
reqd_thermal_df

,Resource,Zone,Model,New_Build,Can_Retire,Existing_Cap_MW,Max_Cap_MW,Min_Cap_MW,Inv_Cost_per_MWyr,Fixed_OM_Cost_per_MWyr,...,Down_Time,Ramp_Up_Percentage,Ramp_Dn_Percentage,Min_Power,Reg_Max,Rsv_Max,Reg_Cost,Rsv_Cost,region,cluster
0,Coal-new,1,1,NaN,NaN,0,-1,0,203928.143456,86500.0,...,9.0,0.292152,0.326875,0.422796,0.25,0.5,0,0,ERCOT,0
1,HTGR,1,1,NaN,NaN,0,-1,0,390650.000000,8200.0,...,20.0,6.000000,6.000000,0.200000,0.25,0.5,0,0,ERCOT,0
3,Microreactor,1,1,NaN,NaN,0,-1,0,545098.500000,13216.5,...,20.0,6.000000,6.000000,0.402985,0.25,0.5,0,0,ERCOT,0
4,NG 2-on-1 Combined Cycle (F-Frame),1,1,NaN,NaN,0,-1,0,74962.652206,34500.0,...,2.0,0.306885,0.341951,0.774372,0.25,0.5,0,0,ERCOT,0
5,NG 2-on-1 Combined Cycle (H-Frame) 95% CCS,1,1,NaN,NaN,0,-1,0,146208.930915,65800.0,...,2.0,0.306885,0.341951,0.774372,0.25,0.5,0,0,ERCOT,0
6,NG Combustion Turbine (F-Frame),1,1,NaN,NaN,0,-1,0,67440.756823,26100.0,...,2.0,0.343447,0.382860,0.597069,0.25,0.5,0,0,ERCOT,0
7,PBR-HTGR,1,1,NaN,NaN,0,-1,0,228450.000000,5000.0,...,20.0,3.000000,3.000000,0.400000,0.25,0.5,0,0,ERCOT,0
8,RICE,1,1,NaN,NaN,0,-1,0,90500.000000,35160.0,...,2.0,0.343447,0.382860,0.500000,0.25,0.5,0,0,ERCOT,0
15,iPWR_Pack,1,1,NaN,NaN,0,-1,0,276751.000000,5750.0,...,20.0,0.400000,0.400000,0.033333,0.25,0.5,0,0,ERCOT,0


In [23]:
# define location of cost assumptions
generator_assumptions_path = os.path.join('..', 'data', 'cases')
# define path locations for CEM and LACs where inputs are going
genx_cem_loc = os.path.join('..', 'GenX.jl', 'research_systems')
spcm_lac_loc = os.path.join('..', 'SPCM', 'research_systems')

# Get the list of all files in the generator_assumptions_path directory
case_names_list = []
for xlsx_name in os.listdir(generator_assumptions_path):
    if os.path.isfile(os.path.join(generator_assumptions_path, xlsx_name)):
        case_name = xlsx_name.replace('.xlsx', '')
        case_names_list.append(case_name)
print(case_names_list)

['10_Hr_BESS', '10_Hr_BESS_Fuelx2', '2_Hr_BESS', '2_Hr_BESS_Fuelx2', '4_Hr_BESS', '4_Hr_BESS_Fuelx2', '4_Hr_BESS_Fuelx3', '4_Hr_BESS_Fuelx4', '6_Hr_BESS', '6_Hr_BESS_Fuelx2', '8_Hr_BESS', '8_Hr_BESS_Fuelx2', 'Thermal_Case']


In [24]:
for case_name in case_names_list:
# for case_name in case_names_list[0:1]:
    # load cem and lac paths
    genx_cem_resources_path = os.path.join(genx_cem_loc, case_name, 'resources')
    spcm_lac_resources_path = os.path.join(spcm_lac_loc, case_name, 'resources')

    # read in cost assumption in generator_assumptions_path for case
    case_assumptions = pd.read_excel(os.path.join(generator_assumptions_path, case_name + '.xlsx'))

    # create a new copy of the reqd_thermal_df for each case
    case_thermal_df = reqd_thermal_df.copy()

    # only take the resources in case_thermal_df that are in case_assumptions
    case_thermal_df = case_thermal_df[case_thermal_df['Resource'].isin(case_assumptions['Technical Name'])]

    for index, row in case_assumptions.iterrows():
        # get the resource name
        resource = row['Technical Name']
        # multiply the cost in case_thermal_df by the corresponding factor in cost_assumptions
        case_thermal_df.loc[case_thermal_df['Resource'] == resource, 'Inv_Cost_per_MWyr'] = \
            case_thermal_df.loc[case_thermal_df['Resource'] == resource, 'Inv_Cost_per_MWyr'] * row['Inv_Cost_per_MWyr_factor']
        case_thermal_df.loc[case_thermal_df['Resource'] == resource, 'Fixed_OM_Cost_per_MWyr'] = \
            case_thermal_df.loc[case_thermal_df['Resource'] == resource, 'Fixed_OM_Cost_per_MWyr'] * row['Fixed_OM_Cost_per_MWyr_factor']
        case_thermal_df.loc[case_thermal_df['Resource'] == resource, 'Var_OM_Cost_per_MWh'] = \
            case_thermal_df.loc[case_thermal_df['Resource'] == resource, 'Var_OM_Cost_per_MWh'] * row['Var_OM_Cost_per_MWh_factor']
        
    # create a cem copy of the case_thermal_df
    cem_case_thermal_df = case_thermal_df.copy()
    # set 'New_Build' to 1
    cem_case_thermal_df['New_Build'] = 1
    # set 'Can_Retire' to -1
    cem_case_thermal_df['Can_Retire'] = 0

    # create a lac copy of the case_thermal_df
    lac_case_thermal_df = case_thermal_df.copy()
    # set 'New_Build' to -1
    lac_case_thermal_df['New_Build'] = -1
    # set 'Can_Retire' to -1
    lac_case_thermal_df['Can_Retire'] = 0

    # save the case_thermal_df to genx_cem_resources_path
    cem_case_thermal_df.to_csv(os.path.join(genx_cem_resources_path, 'Thermal.csv'), index=False)
    # save the case_thermal_df to spcm_lac_resources_path
    lac_case_thermal_df.to_csv(os.path.join(spcm_lac_resources_path, 'Thermal.csv'), index=False)


    # # sort the case_vre_df by 'Resource' alphabetically
    # sorted_case_thermal_df = case_thermal_df.sort_values(by='Resource')
    # sorted_cem_case_thermal_df = cem_case_thermal_df.sort_values(by='Resource')
    # sorted_lac_case_thermal_df = lac_case_thermal_df.sort_values(by='Resource')
    
    # # save the case_thermal_df to genx_cem_resources_path
    # sorted_cem_case_thermal_df.to_csv(os.path.join(genx_cem_resources_path, 'Thermal.csv'), index=False)
    # # save the case_thermal_df to spcm_lac_resources_path
    # sorted_lac_case_thermal_df.to_csv(os.path.join(spcm_lac_resources_path, 'Thermal.csv'), index=False)